In [1]:
import pandas as pd
import numpy as np
import multiprocessing
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
#from gensim.models import word2vec
from inspect import getmembers, isfunction
from math import floor
from catboost import CatBoostRegressor
from rdkit import DataStructs
from rdkit import RDLogger
from sklearn.metrics import mean_squared_error

In [2]:
multiprocessing.cpu_count()

8

In [3]:
RDLogger.DisableLog('rdApp.*')

In [4]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from rdkit.Chem import Descriptors
#from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec

from sklearn.linear_model import RidgeCV

def ecfc_molstring(molecule, radius=1, size=1280):
    arr = np.zeros((1,), dtype=int)
    DataStructs.ConvertToNumpyArray(
        AllChem.GetHashedMorganFingerprint(molecule, radius, size, useFeatures=False, useBondTypes=False,includeRedundantEnvironments=True),
        arr,
    )
    return arr

def ecfc_molstring(molecule, radius=3, size=1024):
    arr = np.zeros((1,), dtype=int)
    DataStructs.ConvertToNumpyArray(
        AllChem.GetHashedMorganFingerprint(molecule, radius, size, useFeatures=False),
        arr,
    )
    return arr

In [5]:
labels = pd.read_csv('./data.csv')
labels.drop(columns=['id'], inplace=True)

In [6]:
df = labels.copy()

In [7]:
def calculate_property(mol):
    output = []
    for i, descriptor in enumerate(descriptor_dict):
        output.append(descriptor_dict[descriptor](mol))
    return output

In [8]:
def number_of_atoms(atom_list, df):
    for i in atom_list:
        df['num_of_{}_atoms'.format(i)] = df['mol'].apply(lambda x: len(x.GetSubstructMatches(Chem.MolFromSmiles(i))))

def prepare_df(df):
    df['mol'] = df['smiles'].apply(lambda x: Chem.MolFromSmiles(x))
    df['mol'] = df['mol'].apply(lambda x: Chem.AddHs(x))
    df['num_of_atoms'] = df['mol'].apply(lambda x: x.GetNumAtoms())
    df['num_of_heavy_atoms'] = df['mol'].apply(lambda x: x.GetNumHeavyAtoms())
    number_of_atoms(['C','O', 'N', 'P','S'], df)

    # df['tpsa'] = df['mol'].apply(lambda x: Descriptors.TPSA(x))
    # df['mol_w'] = df['mol'].apply(lambda x: Descriptors.ExactMolWt(x))
    # df['num_valence_electrons'] = df['mol'].apply(lambda x: Descriptors.NumValenceElectrons(x))
    # df['num_heteroatoms'] = df['mol'].apply(lambda x: Descriptors.NumHeteroatoms(x))
    # df['calc_logp'] = df['mol'].apply(lambda x: Descriptors.MolLogP(x))
    properties = pd.DataFrame([list(Descriptors.CalcMolDescriptors(mol).values()) for mol in (df['mol'])])
    #properties = pd.DataFrame([calculate_property(mol) for mol in tqdm(df['mol'])])
    properties.columns = [f'property_{i}' for i in range(properties.shape[1])]
    #df = pd.concat([df, properties], axis=1)

    # mol_model = word2vec.Word2Vec.load('model_300dim.pkl')
    # mdf = df.copy()
    # mdf['sentence'] = mdf.apply(lambda x: MolSentence(mol2alt_sentence(x['mol'], 1)), axis=1)

    # mdf['mol2vec'] = [DfVec(x) for x in sentences2vec(mdf['sentence'], mol_model, unseen='UNK')]
    # vectors = np.array([x.vec for x in mdf['mol2vec']])
    # vector_df = pd.DataFrame(vectors, columns=[f'mol2vec_feature_{i}' for i in range(vectors.shape[1])])
    # df = pd.concat([df, vector_df], axis=1)
    mols_rdkit = [Chem.MolFromSmiles(smi) for smi in df['smiles']]
    features = np.array([ecfc_molstring(mol) for mol in mols_rdkit])
    df = df.join(pd.DataFrame(features, columns=[f'feature_{i}' for i in range(features.shape[1])]))
    df['space_smiles'] = df['smiles'].apply(lambda x: ' '.join(list(x)))
    #df['ecfc_features'] = list(features)
    return df

In [9]:
df = prepare_df(df)

In [10]:
df.fillna(0, inplace=True)
X = df.drop(columns=['mol', 'lgK', 'smiles'])#[top_features.iloc[:500].feature_names]
y = df['lgK']

In [11]:
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import KFold
N = 5
kf = KFold(n_splits=N, random_state=56, shuffle=True)
models = []
y_pred = pd.Series(np.zeros(y.shape))
for i, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, y_train = X.loc[train_index], y.loc[train_index]
    X_val, y_val = X.loc[test_index], y.loc[test_index]
    model = CatBoostRegressor(
        iterations=1000,
        loss_function='RMSE',
        task_type='CPU',
        verbose=500,
        #grow_policy='Lossguide',
        #learning_rate=0.1,
        random_seed=56,
        max_depth=6,
    )
    model.fit(X_train, y_train, eval_set=(X_val, y_val),text_features=['space_smiles'],)
    y_pred.loc[test_index] = model.predict(X_val)
    models.append(model)
print(mean_squared_error(y, y_pred, squared=False))

Learning rate set to 0.039457
0:	learn: 5.9912122	test: 5.9491106	best: 5.9491106 (0)	total: 51.7ms	remaining: 51.6s
500:	learn: 0.6284027	test: 2.1624640	best: 2.1616017 (498)	total: 1.91s	remaining: 1.9s
999:	learn: 0.2452401	test: 2.1036879	best: 2.1030930 (992)	total: 3.79s	remaining: 0us

bestTest = 2.103092961
bestIteration = 992

Shrink model to first 993 iterations.
Learning rate set to 0.039457
0:	learn: 5.9433366	test: 6.3169694	best: 6.3169694 (0)	total: 4.46ms	remaining: 4.45s
500:	learn: 0.5861394	test: 3.1486853	best: 3.1483053 (467)	total: 1.83s	remaining: 1.83s
999:	learn: 0.2295921	test: 3.1065955	best: 3.1055715 (957)	total: 3.75s	remaining: 0us

bestTest = 3.10557149
bestIteration = 957

Shrink model to first 958 iterations.
Learning rate set to 0.039489
0:	learn: 6.0625430	test: 5.6526894	best: 5.6526894 (0)	total: 4.26ms	remaining: 4.26s
500:	learn: 0.5518125	test: 2.7161355	best: 2.7161355 (500)	total: 1.87s	remaining: 1.86s
999:	learn: 0.2179230	test: 2.6383141	b

In [12]:
history_of_mol = {}

def logK_predict(smiles):
  try:
    if smiles not in history_of_mol:
        data = {'smiles': [smiles]}
        df_test = pd.DataFrame(data)
        df = prepare_df(df_test)
        df.fillna(0, inplace=True)
        X = df.drop(columns=['mol', 'smiles'])

        pred = np.mean([cbm.predict(X) for cbm in models])
        history_of_mol[smiles] = pred

        return pred
    return history_of_mol[smiles]
  except:
    return -1

In [13]:
import selfies as sf

In [27]:
import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Contrib.SA_Score import sascorer

def check_letters(mol):
    mol = Chem.AddHs(mol)
    symbols = [atom.GetSymbol() for atom in mol.GetAtoms()]
    if set(symbols) - set('COHNPS'):
        return False
    if len(set(symbols)) < 3:
        return False
    return symbols.count('O') + symbols.count('N') + symbols.count('P') + symbols.count('S') <= 12

def check_weight(mol):
    return Descriptors.MolWt(mol) <= 500

def check_index(mol):
    return sascorer.calculateScore(mol) < 5

def full_filter(smiles):
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        return False
    
    return check_letters(mol) and check_weight(mol) and check_index(mol)

In [15]:
simple_sol = pd.read_csv("top1000_from130m.csv")

In [16]:
simple_sol['smiles']

0         CC(C)(C)C(CN(CC(=O)O)CC(=O)O)N(CC(=O)O)CC(=O)O
1         CC(=O)CN1CCN(COC=O)CCN(CC(=O)O)CCN(CC(=O)O)CC1
2               C=O.O=C(O)CN(CCN(CC(=O)O)CC(=O)O)CC(=O)O
3        C#CCCCN(CCN(CC(=O)O)CC(=O)O)CCN(CC(=O)O)CC(=O)O
4      CC.CC.CC(=O)CN1CCN(CC(=O)O)CCN(CC(=O)O)CCN(C(C...
                             ...                        
995        CC(=O)CN1CCN(C(=O)O)CCN(C(=O)O)CCN(C(=O)O)CC1
996    CC(=O)NCCN1CCN(CC(=O)O)CCN(CC(=O)O)CCN(CC(=O)O...
997        O=C(O)N1CCCN(C(=O)O)CCN(C(=O)O)CCN(C(=O)O)CC1
998    CCCCNC(=O)CN1CCN(C(=O)O)CCN(C(=O)O)CCN(C(=O)O)CC1
999         O=C(O)N1CCN(C(=O)O)CCN(C(=O)O)CCN(C(=O)O)CC1
Name: smiles, Length: 1000, dtype: object

In [17]:
np.mean([logK_predict(x) for x in tqdm(simple_sol['smiles'])])

  0%|          | 0/1000 [00:00<?, ?it/s]

18.9697973298028

In [28]:
dump_sub = pd.read_csv('./1.csv',names=['smiles'])

In [29]:
tokens = [list(sf.split_selfies(sf.encoder(x))) for x in simple_sol['smiles']] + [list(sf.split_selfies(sf.encoder(x))) for x in dump_sub['smiles']]

In [30]:
tokens = np.concatenate(tokens)

In [31]:
ALPHABET = list(set(tokens))
len(ALPHABET)

32

In [32]:
def to_canonical(smiles):
  return [Chem.MolToSmiles(Chem.MolFromSmiles(m), isomericSmiles=True, canonical=True) for m in smiles]

In [33]:
import random

def delete_aug(mol,num_iters=1, del_step=1):
  mat = list(sf.split_selfies(sf.encoder(mol)))
  for _ in range(num_iters):
    drop_idx = random.randint(0,len(mat)-1)
    mat = mat[:drop_idx] + mat[drop_idx+del_step:]
  return sf.decoder(''.join(mat))

def full_shuffle_aug(mol):
  mat = list(sf.split_selfies(sf.encoder(mol)))
  random.shuffle(mat)
  return sf.decoder(''.join(mat))

def shuffle_aug(mol,num_iters=1):
  mat = list(sf.split_selfies(sf.encoder(mol)))
  N = len(mat)
  for iter in range(num_iters):
    i,j = 0,0
    while i == j:
      i,j = random.randint(0,N-1), random.randint(0,N-1)

    tmp = mat[i]
    mat[i] = mat[j]
    mat[j] = tmp
  return sf.decoder(''.join(mat))

def add_branch_aug(mol,num_iters=1):
  mat = list(sf.split_selfies(sf.encoder(mol)))
  for iter in range(num_iters):
    branch = random.choice(ALPHABET)
    i = random.randint(0,len(mat)-1)
    mat = mat[:i] + [branch] + mat[i:]
  return sf.decoder(''.join(mat))

def replace_branch_aug(mol,num_iters=1):
  mat = list(sf.split_selfies(sf.encoder(mol)))
  for iter in range(num_iters):
    branch = random.choice(ALPHABET)
    i = random.randint(0,len(mat)-1)
    mat = mat[:i] + [branch] + mat[i+1:]
  return sf.decoder(''.join(mat))

def dublicate_aug(mol,num_iters=2):
  mat = list(sf.split_selfies(sf.encoder(mol)))
  mat += mat * num_iters
  return sf.decoder(''.join(mat))

def def_cutout_up_aug(mol,scale=0.5):
  mat = list(sf.split_selfies(sf.encoder(mol)))
  num_del = floor(scale * len(mat))
  mat = mat[:num_del]
  return sf.decoder(''.join(mat))

def def_cutout_pre_aug(mol,scale=0.5):
  mat = list(sf.split_selfies(sf.encoder(mol)))
  num_del = floor(scale * len(mat))
  mat = mat[num_del:]
  return sf.decoder(''.join(mat))

AUGS = [delete_aug,
        full_shuffle_aug,
        shuffle_aug,
        add_branch_aug,
        replace_branch_aug,
        dublicate_aug,
        def_cutout_up_aug,
        def_cutout_pre_aug
        ]

In [34]:
labels.sort_values(by='lgK')[::-1]['smiles']

191    O=P(O)(O)CN(Cc1cccc(CN(CP(=O)(O)O)CP(=O)(O)O)n...
149     O=C(O)N1CCN(CP(=O)(O)O)CCN(C(=O)O)CCN(C(=O)O)CC1
150    O=C(O)CCP(=O)(O)CN1CCN(C(=O)O)CCN(C(=O)O)CCN(C...
136    O=P(O)(O)CN1CCN(CP(=O)(O)O)CCN(CP(=O)(O)O)CCN(...
71     O=C(O)CN1CCN(CC(=O)O)CCN(CC(=O)O)CCN(CC(=O)O)C...
                             ...                        
242                         O=C(O)c1cccc([N+](=O)[O-])c1
42                               O=[N+]([O-])c1ccc(O)cc1
239                                             O=C(O)CI
238                                            O=C(O)CCl
234                                                 O=CO
Name: smiles, Length: 247, dtype: object

In [35]:
simple_sol[::-1]['smiles'][:100]

/tmp/ipykernel_192/686520187.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  simple_sol[::-1]['smiles'][:100]


999         O=C(O)N1CCN(C(=O)O)CCN(C(=O)O)CCN(C(=O)O)CC1
998    CCCCNC(=O)CN1CCN(C(=O)O)CCN(C(=O)O)CCN(C(=O)O)CC1
997        O=C(O)N1CCCN(C(=O)O)CCN(C(=O)O)CCN(C(=O)O)CC1
996    CC(=O)NCCN1CCN(CC(=O)O)CCN(CC(=O)O)CCN(CC(=O)O...
995        CC(=O)CN1CCN(C(=O)O)CCN(C(=O)O)CCN(C(=O)O)CC1
                             ...                        
904    C.O=C(O)CN1CCN(CC(=O)O)CCN(CC(=O)Nc2ccccc2)CCN...
903    CCCCOC(=O)CN1CCN(CC(=O)O)CCN(CC(=O)O)CCN(CC(=O...
902    O=C(O)CN1CCN(CC(=O)O)CCN(Cc2ccccc2O)CCN(CC(=O)...
901      NCCNCCN1CCN(CC(=O)O)CCN(CC(=O)O)CCN(CC(=O)O)CC1
900    O=C(O)CN1CCN(CC(=O)O)CCN(c2ccccc2O)CCN(CC(=O)O...
Name: smiles, Length: 100, dtype: object

In [36]:
temps = labels.sort_values(by='lgK')[::-1]['smiles'][:150].tolist() + simple_sol[::-1]['smiles'][:150].tolist()

/tmp/ipykernel_192/1939323405.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  temps = labels.sort_values(by='lgK')[::-1]['smiles'][:150].tolist() + simple_sol[::-1]['smiles'][:150].tolist()


In [66]:
import re

def get_replace_aug(smiles):
    num_permutations = random.choice([1,1,1,1,1,2,2,3])
    for permute in range(num_permutations):
        gen_set = []
        if '=P' in smiles:
            gen_set += ['P']
        if 'C' in smiles:
            gen_set += ['C']
        if '(=O)' in smiles:
            gen_set += ['O']
        if gen_set == []:
            return smiles
        rep_type = random.choice(gen_set)
        if rep_type == 'O':
            O_idxes = [m.start() for m in re.finditer('(=O)',smiles)]
            per_idx = random.choice(O_idxes)
            per_type = random.choice(['C','P'])
            if per_type == 'C':
                smiles = smiles[:per_idx-1] + 'C' + smiles[per_idx+3:]
            else:
                smiles = smiles[:per_idx-1] + '=P' + smiles[per_idx+3:]
        if rep_type == 'P':
            P_idxes = [m.start() for m in re.finditer('=P',smiles)]
            per_idx = random.choice(P_idxes)
            per_type = random.choice(['C','O'])
            if per_type == 'C':
                smiles = smiles[:per_idx] + 'C' + smiles[per_idx+2:]
            else:
                smiles = smiles[:per_idx] + '(=O)' + smiles[per_idx+2:]
        if rep_type == 'C':
            C_idxes = [m.start() for m in re.finditer('C',smiles)]
            per_idx = random.choice(C_idxes)
            per_type = random.choice(['O','P'])
            if per_type == 'P':
                smiles = smiles[:per_idx] + '=P' + smiles[per_idx+1:]
            else:
                smiles = smiles[:per_idx] + '(=O)' + smiles[per_idx+1:]
    return smiles

In [75]:
def get_augs(temp_mol):
    new_mols = []
    for _ in range(sample_per_temp_mol):
      aug_f = random.choice(AUGS)
      try:
        new_smile = aug_f(temp_mol)
        if full_filter(new_smile):
          new_mols.append(new_smile)
      except:
        pass
    return new_mols

def get_t_augs(temp_mol):
    new_mols = []
    for _ in range(t_sample_per_temp_mol):
        new_smile = get_replace_aug(temp_mol)
        if full_filter(new_smile):
            new_mols.append(new_smile)
    return new_mols

In [76]:
def get_augsV2(temp_mol):
    new_mols = []
    for _ in range(sample_per_temp_mol):
        scale = 2
        aug_f = [random.choice(AUGS) for _ in range(scale)]
        try:
            new_smile = temp_mol
            for f in aug_f:
                new_smile = f(new_smile)
            if full_filter(new_smile):
              new_mols.append(new_smile)
        except:
            pass
    return new_mols

In [77]:
from rdkit import DataStructs

def CalcTanimoto(smiles):
    scores = []
    fpgen = AllChem.GetRDKitFPGenerator()
    fps = [fpgen.GetFingerprint(Chem.MolFromSmiles(x)) for x in smiles]
    for i in range(len(fps)):
        for j in range(i,len(fps)):
            scores += [DataStructs.TanimotoSimilarity(fps[i],fps[j])]
    return np.mean(scores)

In [ ]:
sample_per_temp_mol = 60
t_sample_per_temp_mol = 40
num_iters = 1
avg_meter = {}
pool = multiprocessing.Pool(multiprocessing.cpu_count())

for iter in tqdm(range(num_iters)):
  new_mols = pool.map(get_augs,temps)
  new_mols += pool.map(get_t_augs,temps)
  new_mols = np.concatenate(new_mols).tolist()
  new_mols += temps
  new_mols = to_canonical(new_mols)
  new_mols = np.unique(new_mols)
  #scores = [compute_logP(mol) for mol in new_mols]
  scores = pool.map(logK_predict, tqdm(new_mols))
  #for mol in tqdm(new_mols):
  #  try:
  #    scores.append(logK_predict(mol))
  #  except:
  #    scores.append(-1)

  best_idx = np.argsort(scores)[::-1][:200]
  new_scores = np.sort(scores)[::-1][:200]
  temps = [new_mols[i] for i in best_idx]
  print(np.mean(new_scores[:100]))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/7419 [00:00<?, ?it/s]

In [ ]:
temps

In [82]:
CalcTanimoto(temps)

0.48597775997971054

In [ ]:
pre_temps = simple_sol['smile'].tolist() + dump_sub['smiles'].tolist()

In [83]:
pd.DataFrame(temps[:100]).to_csv('tempsV6.csv',index=False,header=None)

In [2]:
len(temps)

NameError: name 'temps' is not defined

In [ ]:
temps